# Random Forest Algorithm from SCRATCH

### Dataset For training

In [82]:
"""
Color   Diameter    Label

Green   3           Apple
Yellow  3           Apple
Red     1           Grape
Red     1           Grape
Yellow  3           Lemon
"""

'\nColor   Diameter    Label\n\nGreen   3           Apple\nYellow  3           Apple\nRed     1           Grape\nRed     1           Grape\nYellow  3           Lemon\n'

In [83]:
from collections import Counter
import random

In [84]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))


In [85]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [86]:
# Calculate the Gini impurity for a list of rows
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [87]:
# Calculate the information gain
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [88]:
# Find the best question to split the dataset based on information gain
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [89]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)


# Class representing a decision node (question)
class DecisionNode:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [90]:
# Build the decision tree recursively
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    return DecisionNode(question, true_branch, false_branch)


# Print the decision tree
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    print(spacing + str(node.question))

    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "    ")

    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "   ")


# Classify a new input using the decision tree
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions

    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


# Count the occurrences of each class label in the dataset
def class_counts(rows):
    counts = Counter()
    for row in rows:
        label = row[-1]
        counts[label] += 1
    return counts


# Check if a value is numeric
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [91]:
# Random Forest class
class RandomForest:
    def __init__(self, num_trees):
        self.num_trees = num_trees
        self.trees = []

    def fit(self, data):
        for _ in range(self.num_trees):
            # Randomly select a subset of the data
            subset = self.get_random_subset(data)
            tree = build_tree(subset)
            self.trees.append(tree)



# Update the print_leaf function to include probabilities
    def print_leaf(counts):
        total = sum(counts.values()) * 1.0
        probs = {}
        for lbl in counts.keys():
            probs[lbl] = f"{counts[lbl] / total * 100:.0f}%"
        return probs
    
# Update the predict method in the RandomForest class to return predictions with probabilities
    def predict(self, sample):
        predictions = []
        for tree in self.trees:
            prediction = print_leaf(classify(sample, tree))
        predictions.append(prediction)
        return self.get_majority_vote(predictions)


    def get_random_subset(self, data):
        subset = []
        n = len(data)
        for _ in range(n):
            index = random.randint(0, n - 1)
            subset.append(data[index])
        return subset

    def get_majority_vote(self, predictions):
        counts = Counter(tuple(prediction.items()) for prediction in predictions)
        majority_vote = max(counts, key=counts.get)
        return dict(majority_vote)



In [92]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

header = ["color", "diameter", "label"]
my_tree = build_tree(training_data)
print("Decision Tree:")
print_tree(my_tree)
print()

random_forest = RandomForest(num_trees=3)
random_forest.fit(training_data)

print("Random Forest:")
for i, tree in enumerate(random_forest.trees):
    print("Tree", i + 1)
    print_tree(tree)
    print()

Decision Tree:
Is diameter >= 3?
--> True:
    Is color == Yellow?
    --> True:
        Predict Counter({'Apple': 1, 'Lemon': 1})
    --> False:
       Predict Counter({'Apple': 1})
--> False:
   Predict Counter({'Grape': 2})

Random Forest:
Tree 1
Is diameter >= 3?
--> True:
    Is color == Yellow?
    --> True:
        Predict Counter({'Lemon': 1})
    --> False:
       Predict Counter({'Apple': 2})
--> False:
   Predict Counter({'Grape': 2})

Tree 2
Is diameter >= 3?
--> True:
    Is color == Yellow?
    --> True:
        Predict Counter({'Lemon': 2, 'Apple': 1})
    --> False:
       Predict Counter({'Apple': 1})
--> False:
   Predict Counter({'Grape': 1})

Tree 3
Is diameter >= 3?
--> True:
    Is color == Yellow?
    --> True:
        Predict Counter({'Lemon': 1})
    --> False:
       Predict Counter({'Apple': 1})
--> False:
   Predict Counter({'Grape': 3})



In [93]:
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
    ['Red', 1, 'Grape']
    ]

print("Random Forest Predictions:")
for sample in testing_data:
    prediction = random_forest.predict(sample)
    print("Sample:", sample)
    print("Predicted Label:", prediction)
    print()


Random Forest Predictions:
Sample: ['Green', 3, 'Apple']
Predicted Label: {'Apple': '100%'}

Sample: ['Yellow', 4, 'Apple']
Predicted Label: {'Lemon': '100%'}

Sample: ['Red', 2, 'Grape']
Predicted Label: {'Grape': '100%'}

Sample: ['Red', 1, 'Grape']
Predicted Label: {'Grape': '100%'}

Sample: ['Yellow', 3, 'Lemon']
Predicted Label: {'Lemon': '100%'}

Sample: ['Red', 1, 'Grape']
Predicted Label: {'Grape': '100%'}

